# Краулеры

**План**

1. Что такое краулеры?
2. Как написать простой краулер?
3. Блокировки и способы их обхода

## Что такое краулеры?

## Как написать простой краулер?

In [1]:
import requests

In [2]:
session = requests.session()

In [3]:
response = session.get('https://ru.wikipedia.org')

In [4]:
response.headers['X-Client-IP']

'188.32.154.141'

In [5]:
response.headers

{'Date': 'Sun, 18 Aug 2019 14:15:36 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '26224', 'Connection': 'keep-alive', 'Server': 'mw1264.eqiad.wmnet', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Feature-Policy-Report-Only': "sync-xhr 'none'", 'Content-language': 'ru', 'P3P': 'CP="This is not a P3P policy! See https://ru.wikipedia.org/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:CentralAutoLogin/P3P for more info."', 'Origin-Trial': 'AsgJBEgl1uCVxnFLFy3QQSGjCfnGkvAdwJEMdSuoG+rI8vkW1ErSmxh4zE/rAzLcCp+HW7KswymQ6iItKlfihwEAAABqeyJvcmlnaW4iOiJodHRwczovL3J1Lndpa2lwZWRpYS5vcmc6NDQzIiwiZmVhdHVyZSI6IlByaW9yaXR5SGludHMiLCJleHBpcnkiOjE1Njc1NTUxOTksImlzU3ViZG9tYWluIjp0cnVlfQ==, AshJ3chYbVful0vwJ8K3I5UWkQu7KqzD1PZxpgVqCbmWWZz1V3n2jR12Yw+A0Z/GKvsk93s+zgCC7fcN9Q8XDQ0AAABweyJvcmlnaW4iOiJodHRwczovL3J1Lndpa2lwZWRpYS5vcmc6NDQzIiwiZmVhdHVyZSI6IkVsZW1lbnRUaW1pbmdJbWFnZXMiLCJleHBpcnkiOjE1Njc1NTUxOTksImlzU3ViZG9tYWluIjp0cnVlfQ==, AgN+S/0I0APoRAdwjTE11HVRUE4fyP31B11bR7jgVuLPs03/t5

## Блокировки и способы их обхода

Для того, чтобы предотвратить автоматический сбор информации с некого сайта, применяются различные инструменты, которые определяют роботов и блокируют запросы с адресов, которые были классифицированы как роботы. Чтобы не заблокировали домашний/учебный ip, лучше сразу задуматься об этих мерах и предотвратить возможные проблемы. Кстати, Википедия не блокирует и можно спокойно скачивать без каких-либо проблем.

Чтобы их обойти, можно попробовать несколько инструментов:
1. time.sleep(x) - задержка между запросами, чтобы слишком большая скорость запросов не показалась подозрительной или ваши запросы не уронили сервер небольшого ресурса (например, региональной газеты)
2. time.sleep(случайный промежуток времени) - это более хитрая версия, когда время задержки - это случайное число из некоторого отрезка (модуль random)
3. изобразить браузер - при запросе отправляется информация о том, из какого приложения пришел запрос (например, Googlr Chrome), запросы сделанные из браузера больше похожи на человеческие, для этого нужно задать user-agent в параметрах (а его выбирать случайно с помощью fake_useragent)
4. использовать прокси - существуют ресурсы с бесплатными списками открытых прокси, через которые можно пропускать ваш запрос и сервер будет думать, что запросы приходят из разных мест (anonymous и elite классы прокси)

In [6]:
import time

In [7]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(response.headers['Date'])
    time.sleep(3)

Sun, 18 Aug 2019 14:15:36 GMT
Sun, 18 Aug 2019 14:15:39 GMT
Sun, 18 Aug 2019 14:15:42 GMT
Sun, 18 Aug 2019 14:15:45 GMT
Sun, 18 Aug 2019 14:15:49 GMT


In [8]:
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

In [9]:
headers = {'User-Agent': ua.random}
print(headers)
response = session.get('https://ru.wikipedia.org', headers=headers)

{'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36'}


In [10]:
import random

In [11]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(response.headers['Date'])
    time.sleep(random.uniform(1.1, 5.2))

Sun, 18 Aug 2019 14:15:52 GMT
Sun, 18 Aug 2019 14:15:54 GMT
Sun, 18 Aug 2019 14:15:58 GMT
Sun, 18 Aug 2019 14:16:02 GMT
Sun, 18 Aug 2019 14:16:04 GMT


In [13]:
known_proxy_ip = '202.125.75.97:52452'
proxy = {'http': known_proxy_ip, 'https': known_proxy_ip}
response = session.get('https://ru.wikipedia.org', proxies=proxy)
print(response.headers['X-Client-IP'])

202.125.74.6
